# Transform data
Pre-process data records used in original research articles
- assign column types, clean
- rename, reorder columns
- merge tables on keys

In [1]:
import pandas as pd
import numpy as np
import re

Text cleaning helper functions

In [2]:
def remove_nonascii(text):
    return "".join(i for i in text if ord(i)<128)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

**PAPERS** from ICPSR Bibliography (DBInfo)

In [3]:
df_paper = pd.read_excel('../data_original/ICPSR_bib_studies_20211111.xlsx', sheet_name=0)

df_paper = df_paper[df_paper['STUD_NUMS'].notna()] # remove papers that do not have study numbers

df_paper['REF_ID'] = df_paper['REF_ID'].astype('int')
df_paper['TITLE'] = df_paper['TITLE'].str.strip() # remove whitespace from original record entry
df_paper['AUTHORS'] = df_paper['AUTHORS'].str.strip()
df_paper['SEC_TITLE'] = df_paper['SEC_TITLE'].str.strip()
df_paper['RIS_TYPE'] = df_paper['RIS_TYPE'].str.strip()
df_paper['RIS_TYPE'] = df_paper['RIS_TYPE'].astype('category')
df_paper['FUNDER'] = df_paper['FUNDER'].str.strip()
df_paper['YEAR_PUB'] = df_paper['YEAR_PUB'].fillna(0).astype('int')
df_paper['DATE_INPUT'] = df_paper['DATE_INPUT'].astype('datetime64[ns]')
df_paper['STUD_NUMS'] = df_paper['STUD_NUMS'].astype('str')

df_paper.loc[df_paper['YEAR_PUB']<1963,'YEAR_PUB'] = 0  # replace publication year for any item published before ICPSR was established

df_paper = df_paper.rename(columns={'STUD_NUMS':'STUDY_NUMS'})

df_paper = df_paper[['REF_ID',
                     'DOI',
                     'TITLE',
                     'AUTHORS',
                     'SEC_TITLE',
                     'RIS_TYPE',
                     'FUNDER',
                     'YEAR_PUB',
                     'DATE_INPUT',
                     'SERIES_NUMS',
                     'STUDY_NUMS']]

df_paper.to_csv('../data_transform/ICPSR_PAPERS.csv',index=False)
df_paper.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94755 entries, 0 to 99648
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   REF_ID       94755 non-null  int64         
 1   DOI          43063 non-null  object        
 2   TITLE        94750 non-null  object        
 3   AUTHORS      94754 non-null  object        
 4   SEC_TITLE    78193 non-null  object        
 5   RIS_TYPE     94755 non-null  category      
 6   FUNDER       490 non-null    object        
 7   YEAR_PUB     94755 non-null  int64         
 8   DATE_INPUT   94079 non-null  datetime64[ns]
 9   SERIES_NUMS  2338 non-null   object        
 10  STUDY_NUMS   94755 non-null  object        
dtypes: category(1), datetime64[ns](1), int64(2), object(7)
memory usage: 8.0+ MB


In [4]:
df_paper.sample(5)

,REF_ID,DOI,TITLE,AUTHORS,SEC_TITLE,RIS_TYPE,FUNDER,YEAR_PUB,DATE_INPUT,SERIES_NUMS,STUDY_NUMS
5161,5790,NaN,Dimensions of environmental policy support in ...,"Carman, Christopher Jan",Social Science Quarterly,JOUR,NaN,1998,2001-06-06 00:00:00,NaN,06636
61443,114276,10.1111/jomf.12079,Variation in associations between family dinne...,"Meier, Ann; Musick, Kelly",Journal of Marriage and Family,JOUR,NaN,2014,2014-02-10 00:00:00,NaN,21600
82955,140990,10.1007/s10566-018-9483-9,Racial and ethnic differences in teenage fathe...,"Assini-Meytin, Luciana C.; Garza, Mary A.; Gre...",Child and Youth Care Forum,JOUR,NaN,2019,2019-08-19 09:46:01,NaN,21600
34516,43703,NaN,An evaluation of instrumental variable strateg...,"Altonji, Joseph G.; Elder, Todd E.; Taber, Chr...",Journal of Human Resources,JOUR,NaN,2005,2006-07-31 00:00:00,NaN,8085; 9389
12335,14093,NaN,The Media's Role in Forming Voters' National E...,"Hetherington, Marc J.",American Journal of Political Science,JOUR,NaN,1996,2001-04-25 00:00:00,NaN,8298; 9196; 6067


**STUDIES** from ICPSR catalog (DBInfo)

In [5]:
df_study = pd.read_excel('../data_original/ICPSR_bib_studies_20211111.xlsx', sheet_name=1)
df_study = df_study[df_study['PERMIT']=='AVAILABLE'] # remove studies that are not publicly available
df_study = df_study[df_study['OBJECTTYPE']=='study'] # remove union catalog entries

df_study['DESCRIPTION'] = df_study['DESCRIPTION_1'].astype('str') \
    + " " + df_study['DESCRIPTION_2'].astype('str') \
    + " " + df_study['DESCRIPTION_3'].astype('str') + " " \
    + df_study['DESCRIPTION_4'].astype('str') \
    + " " + df_study['DESCRIPTION_5'].astype('str') # combine study description to a single field

df_study = df_study.drop(columns=['PERMIT',
                                  'OBJECTTYPE',
                                  'ALTTITLE1',
                                  'ALTTITLE2',
                                  'ALTTITLE3',
                                  'ALTTITLE4',
                                  'ALTTITLE5',
                                  'ALTTITLE6',
                                  'ALTTITLE7',
                                  'DESCRIPTION_1',
                                  'DESCRIPTION_2',
                                  'DESCRIPTION_3',
                                  'DESCRIPTION_4',
                                  'DESCRIPTION_5'])

df_study['STUDY'] = df_study['STUDY'].astype('int')
df_study['NAME'] = df_study['NAME'].str.strip()
df_study['SERIES'] = df_study['SERIES'].astype('str').astype('float')
df_study['SERIES_TITLE'] = df_study['SERIES_TITLE'].str.strip()
df_study['OWNER'] = df_study['OWNER'].astype('category')
df_study['FUNDINGAGENCY'] = df_study['FUNDINGAGENCY'].str.strip()
df_study['DOI'] = df_study['DOI'].str.strip()
df_study['GEO'] = df_study['GEO'].str.strip()
df_study['TERMS'] = df_study['TERMS'].str.strip()
df_study['ORIGRELDATE'] = df_study['ORIGRELDATE'].fillna(0).astype('datetime64[ns]')
df_study['MEMSERV_PI'] = df_study['MEMSERV_PI'].str.strip()
df_study['DESCRIPTION'] = df_study['DESCRIPTION'].str.strip().astype('str')

df_study['DESCRIPTION'] = df_study['DESCRIPTION'].apply(func=remove_nonascii) # remove nonascii
df_study['DESCRIPTION'] = df_study['DESCRIPTION'].apply(func=remove_html) # remove html tags
df_study['DESCRIPTION'] = df_study['DESCRIPTION'].replace(regex=["nan"], value="") # remove string "nan"

df_study = df_study.rename(columns={'FUNDINGAGENCY':'FUNDING_AGENCY','ORIGRELDATE':'RELEASE_DATE','MEMSERV_PI':'PRINCIPAL_INV'})
df_study.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10610 entries, 0 to 11660
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   STUDY           10610 non-null  int64         
 1   NAME            10610 non-null  object        
 2   SERIES          6683 non-null   float64       
 3   SERIES_TITLE    6683 non-null   object        
 4   OWNER           10610 non-null  category      
 5   FUNDING_AGENCY  5256 non-null   object        
 6   DOI             10610 non-null  object        
 7   GEO             9956 non-null   object        
 8   TERMS           10155 non-null  object        
 9   RELEASE_DATE    10610 non-null  datetime64[ns]
 10  PRINCIPAL_INV   10610 non-null  object        
 11  DESCRIPTION     10610 non-null  object        
dtypes: category(1), datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 1005.8+ KB


Study table: usage

In [6]:
df_usage = pd.read_excel('../data_original/ICPSR_combined_study_usage_20210430_onlyData_noAllREST.xlsx',sheet_name=0)
df_usage = df_usage.drop(columns=['DATA_PULL_DATE','ORIGRELDATE','OWNER','OWNER_ICPSR','MEMBER','DAYSINSAMPLE_TO20210430','DAYSINSAMPLE_TO20151231','RECENCY','SERIESYN','VARS','SAMPLING','PROPORTIONREST','TOT_DATA', 'NUMTERMS'])

df_usage['STUDY'] = df_usage['STUDY'].astype('int')
df_usage['SINGLEPI'] = df_usage['SINGLEPI'].astype('int')
df_usage['INST_PI'] = df_usage['INST_PI'].astype('int')
df_usage['TOT_PI'] = df_usage['TOT_PI'].astype('int')
df_usage['SDA'] = df_usage['SDA'].astype('int')
df_usage['QTEXT'] = df_usage['QTEXT'].astype('float')
df_usage['SSVD'] = df_usage['SSVD'].astype('float')
df_usage['USERS_2017_TO_PULLDATE'] = df_usage['USERS_2017_TO_PULLDATE'].astype('float')
df_usage['DATAUSERS_2017_TO_PULLDATE'] = df_usage['DATAUSERS_2017_TO_PULLDATE'].astype('float')
df_usage['HAS_RESTRICTED'] = df_usage['HAS_RESTRICTED'].astype('int')
df_usage['ALL_RESTRICTED'] = df_usage['ALL_RESTRICTED'].astype('int')

df_usage = df_usage.rename(columns={'USERS_2017_TO_PULLDATE':'USERS_TO_20210511','DATAUSERS_2017_TO_PULLDATE':'DATAUSERS_TO_20210511','SINGLEPI':'SINGLE_PI'})
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   STUDY                  461 non-null    int64  
 1   SINGLE_PI              461 non-null    int64  
 2   INST_PI                461 non-null    int64  
 3   TOT_PI                 461 non-null    int64  
 4   SDA                    461 non-null    int64  
 5   QTEXT                  453 non-null    float64
 6   SSVD                   453 non-null    float64
 7   USERS_TO_20210511      457 non-null    float64
 8   DATAUSERS_TO_20210511  455 non-null    float64
 9   HAS_RESTRICTED         461 non-null    int64  
 10  ALL_RESTRICTED         461 non-null    int64  
dtypes: float64(4), int64(7)
memory usage: 39.7 KB


Study table: curation level

In [7]:
df_curation = pd.read_csv('../data_original/jira_curation_level_20210426_by_study.csv')
df_curation = df_curation.drop(columns=['cur_lev_rank'])
df_curation['study'] = df_curation['study'].astype('int')
df_curation['curation_level'] = df_curation['curation_level'].astype('category')
df_curation = df_curation.rename(columns={'study':'STUDY','curation_level':'CURATION_LEVEL'})
df_curation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1367 entries, 0 to 1366
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   STUDY           1367 non-null   int64   
 1   CURATION_LEVEL  1367 non-null   category
dtypes: category(1), int64(1)
memory usage: 12.4 KB


Study table: citations (count number of study citations from Bibliography)

In [8]:
df_paper['STUDY'] = df_paper['STUDY_NUMS'].str.split(";") # split STUD_NUMS so that each row is one study
df_paper = df_paper.explode('STUDY')
df_paper['STUDY'] = df_paper['STUDY'].astype('int')

df_citations = pd.DataFrame(df_paper['STUDY'].value_counts()).reset_index()
df_citations = df_citations.rename(columns={'count':'CITATIONS_TO_20211116'})
df_citations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8377 entries, 0 to 8376
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   STUDY                  8377 non-null   int64
 1   CITATIONS_TO_20211116  8377 non-null   int64
dtypes: int64(2)
memory usage: 131.0 KB


Study table: variable counts

In [9]:
df_variables = pd.read_csv('../data_original/variable_counts_by_study.csv')
df_variables['STUDY'] = df_variables['STUDY'].astype('int')
df_variables['TOTALVARS'] = df_variables['TOTALVARS'].astype('int')
df_variables = df_variables.rename(columns={'TOTALVARS':'TOTAL_VARS'})
df_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9485 entries, 0 to 9484
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   STUDY       9485 non-null   int64
 1   TOTAL_VARS  9485 non-null   int64
dtypes: int64(2)
memory usage: 148.3 KB


Merge STUDY tables

In [10]:
df_study_merge = df_study.merge(df_usage, on='STUDY', how='left') \
    .merge(df_curation, on='STUDY', how='left') \
    .merge(df_citations, on='STUDY', how='left') \
    .merge(df_variables, on='STUDY', how='left') # merge all tables on main list of studies

df_study_merge = df_study_merge[['STUDY', 
                               'DOI',
                               'NAME',
                               'SERIES',
                               'SERIES_TITLE',
                               'PRINCIPAL_INV',
                               'DESCRIPTION',
                               'RELEASE_DATE',
                               'FUNDING_AGENCY',
                               'GEO',
                               'TERMS',
                               'OWNER',
                               'CURATION_LEVEL',
                               'SINGLE_PI',
                               'INST_PI',
                               'TOT_PI',
                               'TOTAL_VARS',
                               'SDA',
                               'QTEXT',
                               'SSVD',
                               'HAS_RESTRICTED',
                               'ALL_RESTRICTED',
                               'USERS_TO_20210511',
                               'DATAUSERS_TO_20210511',
                               'CITATIONS_TO_20211116']]

df_study_merge.to_csv('../data_transform/ICPSR_STUDIES.csv',index=False)
df_study_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10610 entries, 0 to 10609
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   STUDY                  10610 non-null  int64         
 1   DOI                    10610 non-null  object        
 2   NAME                   10610 non-null  object        
 3   SERIES                 6683 non-null   float64       
 4   SERIES_TITLE           6683 non-null   object        
 5   PRINCIPAL_INV          10610 non-null  object        
 6   DESCRIPTION            10610 non-null  object        
 7   RELEASE_DATE           10610 non-null  datetime64[ns]
 8   FUNDING_AGENCY         5256 non-null   object        
 9   GEO                    9956 non-null   object        
 10  TERMS                  10155 non-null  object        
 11  OWNER                  10610 non-null  category      
 12  CURATION_LEVEL         1125 non-null   category      
 13  S

In [11]:
df_study_merge.sample(5)

,STUDY,DOI,NAME,SERIES,SERIES_TITLE,PRINCIPAL_INV,DESCRIPTION,RELEASE_DATE,FUNDING_AGENCY,GEO,...,TOT_PI,TOTAL_VARS,SDA,QTEXT,SSVD,HAS_RESTRICTED,ALL_RESTRICTED,USERS_TO_20210511,DATAUSERS_TO_20210511,CITATIONS_TO_20211116
9329,36356,10.3886/ICPSR36356.v1,"Bridging the Gap, Elementary School Data (Food...",NaN,NaN,"Chaloupka, Frank J.",The Food and Fitness Survey is part of the lar...,2018-01-15 11:53:56,Robert Wood Johnson Foundation,United States,...,1.0,7681.0,1.0,0.0,1.0,1.0,0.0,221.0,162.0,18.0
975,2711,10.3886/ICPSR02711.v1,"Washington Post 'What Clinton Should Do' Poll,...",1.0,ABC News/Washington Post Poll Series,THE WASHINGTON POST.,"This special topic poll, fielded December 15, ...",1999-06-23 00:00:00,NaN,United States,...,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579,2238,10.3886/ICPSR02238.v1,Elementary and Secondary General Information S...,101.0,Elementary & Secondary Education General Infor...,United States Department of Education. Nation...,This dataset contains records for each public ...,2000-08-28 00:00:00,NaN,American Samoa; Guam; Marshall Islands; Puerto...,...,NaN,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2902,6023,10.3886/ICPSR06023.v1,"ABC News Vice-Presidential Debate Poll, Octobe...",1.0,ABC News/Washington Post Poll Series,ABC News,"In this poll, taken after the vice-presidentia...",1993-10-11 00:00:00,NaN,United States,...,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1786,3606,10.3886/ICPSR03606.v1,"Survey of Consumer Finances, 1949",55.0,Survey of Consumer Finances Series,Economic Behavior Program. Survey Research Ce...,This data collection is one in a series of fic...,1984-05-11 00:00:00,National Science Foundation,United States,...,NaN,-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


**PREDICTED CURATION ACTIVITIES** from Jira work logs

In [12]:
df_action = pd.read_csv('../data_original/predicted_curation_by_study.csv')
df_action = df_action.drop(columns=['Unnamed: 0','Desc']) # remove sentence index and work descriptions (identifiable)
df_action = df_action.rename(columns={'Studies':'STUDY','Action':'ACTION_LABEL','Log_hrs':'LOG_HRS','Study_hrs':'STUDY_HRS'})
df_action['STUDY'] = df_action['STUDY'].str.replace("s","") # remove "s" from study number
df_action['STUDY'] = df_action['STUDY'].str.strip()
df_action['STUDY'] = df_action['STUDY'].astype('int')
df_action['ACTION_LABEL'] = df_action['ACTION_LABEL'].str.strip()
df_action['ACTION_LABEL'] = df_action['ACTION_LABEL'].astype('category')
df_action.to_csv('../data_transform/ICPSR_CURATION_LOGS.csv',index=False)
df_action.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13425 entries, 0 to 13424
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   STUDY         13425 non-null  int64   
 1   ACTION_LABEL  13425 non-null  category
 2   LOG_HRS       12216 non-null  float64 
 3   STUDY_HRS     13425 non-null  float64 
dtypes: category(1), float64(2), int64(1)
memory usage: 328.2 KB


In [13]:
df_action.head(5)

,STUDY,ACTION_LABEL,LOG_HRS,STUDY_HRS
0,37216,Metadata-study-level,6.5,712.5
1,37216,Quality-checks,1.0,712.5
2,37216,Quality-checks,8.0,712.5
3,37216,Metadata-study-level,4.0,712.5
4,37216,Non-curation,7.0,712.5


**PROCESSING HISTORY COMMANDS** from SPSS processing history files

Will not include in deposit

In [14]:
# df_processing = pd.read_csv('../data_original/processing_history_commands_2019_20220502.csv')
# df_processing = df_processing.drop(columns=['filename','savedate'])
# df_processing.columns = [x.upper() for x in df_processing.columns]
# df_processing['STUDY'] = df_processing['STUDY'].astype('int')
# df_processing['TOTAL_LINES'] = df_processing['TOTAL_LINES'].astype('int')
# df_processing['COMMENTS'] = df_processing['COMMENTS'].astype('int')
# df_processing.to_csv('../data_transform/ICPSR_PROCESSING_HISTORY.csv',index=False)

# for column in df_processing.columns:
#     print(column)

In [15]:
# df_processing.sample(5)